## Uvod

Ova lekcija obuhvaća:
- Što je pozivanje funkcija i kada se koristi
- Kako napraviti poziv funkcije pomoću Azure OpenAI
- Kako integrirati poziv funkcije u aplikaciju

## Ciljevi učenja

Nakon završetka ove lekcije znat ćete i razumjeti:

- Svrhu korištenja pozivanja funkcija
- Postavljanje poziva funkcije pomoću Azure Open AI servisa
- Dizajniranje učinkovitih poziva funkcija za potrebe vaše aplikacije


## Razumijevanje poziva funkcija

Za ovu lekciju želimo izgraditi funkcionalnost za naš startup u obrazovanju koja korisnicima omogućuje korištenje chatbota za pronalaženje tehničkih tečajeva. Preporučit ćemo tečajeve koji odgovaraju njihovoj razini znanja, trenutnoj ulozi i tehnologiji koja ih zanima.

Za ostvarenje ovoga koristit ćemo kombinaciju:
 - `Azure Open AI` za kreiranje chat iskustva za korisnika
 - `Microsoft Learn Catalog API` kako bismo korisnicima pomogli pronaći tečajeve prema njihovom zahtjevu
 - `Function Calling` kako bismo korisnički upit poslali funkciji koja će napraviti API zahtjev.

Za početak, pogledajmo zašto bismo uopće željeli koristiti pozivanje funkcija:

print("Poruke u sljedećem zahtjevu:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dobij novu GPT-ovu reakciju gdje može vidjeti odgovor funkcije


print(second_response.choices[0].message)


### Zašto koristiti Function Calling

Ako ste završili bilo koju drugu lekciju u ovom tečaju, vjerojatno ste već upoznati s moći korištenja velikih jezičnih modela (LLM-ova). Nadamo se da ste primijetili i neka njihova ograničenja.

Function Calling je značajka Azure Open AI servisa koja rješava sljedeća ograničenja:
1) Dosljedan format odgovora
2) Mogućnost korištenja podataka iz drugih izvora aplikacije u kontekstu razgovora

Prije function callinga, odgovori iz LLM-a bili su nestrukturirani i nedosljedni. Programeri su morali pisati složen kod za provjeru valjanosti kako bi mogli obraditi svaku varijaciju odgovora.

Korisnici nisu mogli dobiti odgovore poput "Kakvo je trenutno vrijeme u Stockholmu?". To je zato što su modeli bili ograničeni na razdoblje u kojem su podaci trenirani.

Pogledajmo primjer ispod koji ilustrira ovaj problem:

Recimo da želimo napraviti bazu podataka o studentima kako bismo im mogli predložiti odgovarajući tečaj. Ispod imamo dva opisa studenata koji su vrlo slični po podacima koje sadrže.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Želimo ovo poslati LLM-u kako bi obradio podatke. Kasnije to možemo koristiti u našoj aplikaciji za slanje API-ju ili pohranu u bazu podataka.

Napravimo dva identična upita kojima ćemo uputiti LLM na informacije koje nas zanimaju:


Želimo ovo poslati LLM-u kako bi analizirao dijelove koji su važni za naš proizvod. Tako možemo stvoriti dva identična upita za davanje uputa LLM-u:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Nakon što izradimo ova dva upita, poslat ćemo ih LLM-u koristeći `openai.ChatCompletion`. Upit pohranjujemo u varijablu `messages` i dodjeljujemo ulogu `user`. Ovo je kako bismo oponašali poruku korisnika upućenu chatbotu.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Sada možemo poslati oba zahtjeva LLM-u i ispitati odgovor koji primimo.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Iako su upiti isti i opisi slični, možemo dobiti različite formate svojstva `Grades`.

Ako pokrenete gornju ćeliju više puta, format može biti `3.7` ili `3.7 GPA`.

To je zato što LLM prima nestrukturirane podatke u obliku pisanog upita i također vraća nestrukturirane podatke. Potreban nam je strukturirani format kako bismo znali što očekivati prilikom pohrane ili korištenja tih podataka.

Korištenjem funkcijskog pozivanja možemo osigurati da dobijemo strukturirane podatke natrag. Kod funkcijskog pozivanja, LLM zapravo ne poziva niti pokreće nikakve funkcije. Umjesto toga, stvaramo strukturu koju LLM treba slijediti u svojim odgovorima. Zatim koristimo te strukturirane odgovore kako bismo znali koju funkciju pokrenuti u našim aplikacijama.


![Dijagram toka poziva funkcije](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.hr.png)


### Primjeri korištenja poziva funkcija

**Pozivanje vanjskih alata**  
Chatbotovi su izvrsni u davanju odgovora na korisnička pitanja. Korištenjem poziva funkcija, chatbotovi mogu koristiti poruke korisnika za izvršavanje određenih zadataka. Na primjer, student može zamoliti chatbot da "Pošalji e-mail mom profesoru i reci da mi treba dodatna pomoć s ovim predmetom". Ovo može napraviti poziv funkciji `send_email(to: string, body: string)`

**Izrada API ili baza podataka upita**  
Korisnici mogu pronaći informacije koristeći prirodni jezik koji se pretvara u formatirani upit ili API zahtjev. Primjer za to može biti učitelj koji pita "Tko su učenici koji su završili posljednji zadatak", što može pozvati funkciju pod nazivom `get_completed(student_name: string, assignment: int, current_status: string)`

**Stvaranje strukturiranih podataka**  
Korisnici mogu uzeti blok teksta ili CSV i koristiti LLM za izdvajanje važnih informacija iz njega. Na primjer, student može pretvoriti članak s Wikipedije o mirovnim sporazumima kako bi napravio AI kartice za učenje. Ovo se može napraviti korištenjem funkcije `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Kreiranje vašeg prvog poziva funkcije

Proces kreiranja poziva funkcije uključuje 3 glavna koraka:
1. Pozivanje Chat Completions API-ja s popisom vaših funkcija i korisničkom porukom
2. Čitanje odgovora modela kako biste poduzeli neku radnju, npr. izvršili funkciju ili API poziv
3. Ponovno pozivanje Chat Completions API-ja s odgovorom vaše funkcije kako biste iskoristili te informacije za kreiranje odgovora korisniku.


![Tijek poziva funkcije](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.hr.png)


### Elementi poziva funkcije

#### Korisnički unos

Prvi korak je kreirati korisničku poruku. To se može dinamički dodijeliti uzimanjem vrijednosti iz tekstualnog unosa ili možete ovdje ručno postaviti vrijednost. Ako prvi put radite s Chat Completions API-jem, potrebno je definirati `role` i `content` poruke.

`role` može biti `system` (postavljanje pravila), `assistant` (model) ili `user` (krajnji korisnik). Za pozivanje funkcije, ovo ćemo postaviti kao `user` i dodati primjer pitanja.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Kreiranje funkcija.

Sljedeće ćemo definirati funkciju i njezine parametre. Ovdje ćemo koristiti samo jednu funkciju pod nazivom `search_courses`, ali možete napraviti više funkcija.

**Važno**: Funkcije su uključene u sistemsku poruku prema LLM-u i ulaze u ukupan broj dostupnih tokena koje imate na raspolaganju.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicije**

`name` - Naziv funkcije koju želimo pozvati.

`description` - Opis načina na koji funkcija radi. Ovdje je važno biti precizan i jasan.

`parameters` - Popis vrijednosti i formata koje želite da model generira u svom odgovoru.

`type` - Tip podataka u kojem će svojstva biti pohranjena.

`properties` - Popis konkretnih vrijednosti koje će model koristiti za svoj odgovor.

`name` - Naziv svojstva koje će model koristiti u svom formatiranom odgovoru.

`type` - Tip podataka ovog svojstva.

`description` - Opis konkretnog svojstva.

**Opcionalno**

`required` - Obavezno svojstvo da bi se poziv funkcije mogao izvršiti.


### Pozivanje funkcije
Nakon što smo definirali funkciju, sada je trebamo uključiti u poziv prema Chat Completion API-ju. To radimo tako da dodamo `functions` u zahtjev. U ovom slučaju `functions=functions`.

Postoji i opcija da postavimo `function_call` na `auto`. To znači da ćemo prepustiti LLM-u da odluči koju funkciju treba pozvati na temelju korisničke poruke, umjesto da to sami određujemo.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sada pogledajmo odgovor i provjerimo kako je formatiran:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Možeš vidjeti da je pozvana funkcija po imenu, a iz korisničke poruke LLM je uspio pronaći podatke koji odgovaraju argumentima funkcije.


## 3. Integracija poziva funkcija u aplikaciju.

Nakon što smo testirali formatirani odgovor iz LLM-a, sada to možemo integrirati u aplikaciju.

### Upravljanje tokom

Da bismo ovo integrirali u našu aplikaciju, slijedimo sljedeće korake:

Prvo, pozovimo Open AI servise i spremimo poruku u varijablu pod nazivom `response_message`.


In [ ]:
response_message = response.choices[0].message

Sada ćemo definirati funkciju koja će pozvati Microsoft Learn API kako bi dohvatila popis tečajeva:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kao najbolju praksu, prvo ćemo provjeriti želi li model pozvati neku funkciju. Nakon toga, izradit ćemo jednu od dostupnih funkcija i povezati je s funkcijom koju model želi pozvati.
Zatim ćemo uzeti argumente funkcije i mapirati ih na argumente iz LLM-a.

Na kraju, dodat ćemo poruku o pozivu funkcije i vrijednosti koje su vraćene porukom `search_courses`. Ovo daje LLM-u sve potrebne informacije kako bi mogao odgovoriti korisniku na prirodan način.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Izazov s kodom

Odlično odrađeno! Za nastavak učenja o Azure Open AI Function Calling možete izraditi: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Više parametara funkcije koji mogu pomoći polaznicima da pronađu više tečajeva. Dostupne API parametre možete pronaći ovdje:
 - Izradite još jedan poziv funkcije koji uzima više informacija od polaznika, poput njihovog materinjeg jezika
 - Dodajte obradu grešaka kada poziv funkcije i/ili API-ja ne vrati nijedan odgovarajući tečaj



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge prevođenja [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
